In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import math
import statistics
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import regex as re

In [2]:
# Read Muddy Paws csv data file and assign it to a variable as a dataframe
muddy_p_df = pd.read_excel('part_1_mp_cleaning.xlsx')

In [3]:
# Check the column names to see if any need to be dropped
muddy_p_df.columns

Index(['Unnamed: 0', 'pet_name', 'pet_breed', 'other_name', 'litter_name',
       'parent_of_litter', 'date_pet_entered_your_care', 'length_of_stay_days',
       'adoption_foster_date', 'current_status', 'pet_type', 'pet_age',
       'petstablished_id', 'size', 'weight', 'color', 'shedding',
       'coat_length', 'temperament', 'breed_type',
       'date_placed_in_current_location', 'microchip_id',
       'microchip_manufacturer', 'adoption_fee', 'internal_id',
       'animal_control_id', 'adoption_application_visible',
       'foster_application_visible', 'last_updated_at', 'gender',
       'acquired_by', 'date_of_birth', 'age_in_years', 'declawed_status',
       'coat_pattern', 'events_attendance', 'is_mix', 'shots_up_to_date',
       'spayed_neutered', 'hypoallergenic', 'housebroken', 'special_need',
       'needs_foster', 'gets_along_with_cats', 'gets_along_with_dogs',
       'gets_along_with_kids', 'description', 'internal_notes',
       'behavioral_tracking_notes', 'has_finalized

In [4]:
# Drop the irrelevant columns (mostly related to applications and fosters)
muddy_p_df = muddy_p_df.drop(['adoption_application_visible', 'foster_application_visible', 'needs_foster', 'has_finalized_application', 
                'date_pet_entered_your_care_1'], axis = 1)

In [5]:
# Check the columns to ensure that the correct ones were dropped
muddy_p_df.columns

Index(['Unnamed: 0', 'pet_name', 'pet_breed', 'other_name', 'litter_name',
       'parent_of_litter', 'date_pet_entered_your_care', 'length_of_stay_days',
       'adoption_foster_date', 'current_status', 'pet_type', 'pet_age',
       'petstablished_id', 'size', 'weight', 'color', 'shedding',
       'coat_length', 'temperament', 'breed_type',
       'date_placed_in_current_location', 'microchip_id',
       'microchip_manufacturer', 'adoption_fee', 'internal_id',
       'animal_control_id', 'last_updated_at', 'gender', 'acquired_by',
       'date_of_birth', 'age_in_years', 'declawed_status', 'coat_pattern',
       'events_attendance', 'is_mix', 'shots_up_to_date', 'spayed_neutered',
       'hypoallergenic', 'housebroken', 'special_need', 'gets_along_with_cats',
       'gets_along_with_dogs', 'gets_along_with_kids', 'description',
       'internal_notes', 'behavioral_tracking_notes',
       'where_was_pet_originally_found', 'coordinators', 'additional_comments',
       'owner_requested_eu

Check length of stay for valid data.

In [6]:
# Check the minimum value in length of stay
muddy_p_df.length_of_stay_days.min()

-2457686.0

In [7]:
# Get rid of rows containing values less than 0 in length of stay column
muddy_p_df.length_of_stay_days = muddy_p_df.length_of_stay_days[muddy_p_df.length_of_stay_days >= 0]

In [8]:
# Check the minimum value in length of stay again
muddy_p_df.length_of_stay_days.min()

0.0

In [9]:
# Check the maximum value in length of stay
muddy_p_df.length_of_stay_days.max()

2158.0

I want to remove all animals that are not breeds of dog in order to do specific analysis on dogs.

In [10]:
# Check how many of each pet_type there are - this will help if I decide to do future analysis on other animal types
muddy_p_df.pet_type.value_counts()

dog              3007
cat               442
small animal       60
rabbit             29
other               9
reptile, fish       6
bird                5
Name: pet_type, dtype: int64

In [11]:
# Remove all pet types that aren't dogs
muddy_p_df = muddy_p_df[muddy_p_df.pet_type.str.contains('cat|small animal|rabbit|other|reptile, fish|bird') == False]

In [12]:
# Check the pet type column to ensure that dog is the only type
muddy_p_df.pet_type.value_counts()

dog    3007
Name: pet_type, dtype: int64

I need to remove the extra dog breed descriptions from the pet_breed column in order to isolate primary breed type.

In [13]:
# Split the pet_breed column on the , to turn it into a list
muddy_p_df['pet_breed'] = muddy_p_df['pet_breed'].str.split(', ', expand = True)

In [14]:
# View the number of entries for each isolated breed
pd.set_option('display.max_rows', None)
muddy_p_df.pet_breed.value_counts()

labrador retriever                 383
chihuahua                          203
german shepherd dog                170
pit bull terrier                   166
boxer                              141
yorkie                             119
shih tzu                            96
beagle                              81
australian shepherd                 81
maltese                             59
border collie                       53
pomeranian                          49
31 flavors mixes                    48
shepherds (all types)               46
dachshund                           44
great pyrenees                      43
husky                               39
blue heeler                         35
terrier (unknown type               33
australian cattle dog               33
pug                                 31
jack russell terrier                28
rat terrier                         26
miniature pinscher                  25
corgi                               24
terriers (small)         

In [15]:
# Remove any breeds with less than 10 animals listed

# Instantiate a list containing the value counts for each breed
counts = muddy_p_df.pet_breed.value_counts()
# Remove the pet_breed row if there are less than 10 animals in the counts list
muddy_p_df = muddy_p_df[~muddy_p_df.pet_breed.isin(counts[counts < 10].index)]

In [16]:
# Check the value counts
muddy_p_df.pet_breed.value_counts()

labrador retriever                383
chihuahua                         203
german shepherd dog               170
pit bull terrier                  166
boxer                             141
yorkie                            119
shih tzu                           96
beagle                             81
australian shepherd                81
maltese                            59
border collie                      53
pomeranian                         49
31 flavors mixes                   48
shepherds (all types)              46
dachshund                          44
great pyrenees                     43
husky                              39
blue heeler                        35
terrier (unknown type              33
australian cattle dog              33
pug                                31
jack russell terrier               28
rat terrier                        26
miniature pinscher                 25
corgi                              24
cocker spaniel                     23
terriers (sm

We have a problem in that several different breed names are used for the same type of dog (American Staffordshire Terrier). I want to combine those into one breed type.

In [17]:
# Use .replace to replace each name with the name american staffordshire terrier
muddy_p_df.pet_breed = muddy_p_df.pet_breed.replace({'pit bull terrier':'american staffordshire terrier', 'pit bull':'american staffordshire terrier', 
               'pittie':'american staffordshire terrier', 'pitbull':'american staffordshire terrier', 
                'bull terrier':'american staffordshire terrier', 'pit bull terrier   ':'american staffordshire terrier', 
               'american pit bull terrier':'american staffordshire terrier', 'staffordshire bull terrier':'american staffordshire terrier'})

In [18]:
# Check the breed value counts to ensure that only american staffordshire terrier is included - the number in that category
# should go up
muddy_p_df.pet_breed.value_counts()

labrador retriever                383
american staffordshire terrier    212
chihuahua                         203
german shepherd dog               170
boxer                             141
yorkie                            119
shih tzu                           96
beagle                             81
australian shepherd                81
maltese                            59
border collie                      53
pomeranian                         49
31 flavors mixes                   48
shepherds (all types)              46
dachshund                          44
great pyrenees                     43
husky                              39
blue heeler                        35
terrier (unknown type              33
australian cattle dog              33
pug                                31
jack russell terrier               28
rat terrier                        26
miniature pinscher                 25
corgi                              24
cocker spaniel                     23
terriers (sm

In [19]:
# This helps me see every column in the dataframe easily
pd.set_option('display.max_columns', None)

In [20]:
# Check the top of the dataframe to get a sense for what we're working with
muddy_p_df.head()

,Unnamed: 0,pet_name,pet_breed,other_name,litter_name,parent_of_litter,date_pet_entered_your_care,length_of_stay_days,adoption_foster_date,current_status,pet_type,pet_age,petstablished_id,size,weight,color,shedding,coat_length,temperament,breed_type,date_placed_in_current_location,microchip_id,microchip_manufacturer,adoption_fee,internal_id,animal_control_id,last_updated_at,gender,acquired_by,date_of_birth,age_in_years,declawed_status,coat_pattern,events_attendance,is_mix,shots_up_to_date,spayed_neutered,hypoallergenic,housebroken,special_need,gets_along_with_cats,gets_along_with_dogs,gets_along_with_kids,description,internal_notes,behavioral_tracking_notes,where_was_pet_originally_found,coordinators,additional_comments,owner_requested_euthanasia,additional_internal_comments,type_of_intake,siblings,courtesy_post,cross_post
5,5,aaliyah,australian shepherd,,NaN,NaN,2023-01-08,8.0,2023-01-16,adopted,dog,puppy,ps1615513,medium,NaN,"gray, black, white",sheds a lot,NaN,NaN,NaN,2023-01-17,900085001279038,NaN,600,NaN,NaN,2023-01-17,female,NaN,2023-11-04,-283 days,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,<p>are you looking for an active pup to take w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relinquished by owner,NaN,0,0
6,6,abbey,schnauzer (standard),,NaN,NaN,2021-06-02,43.0,2021-07-15,adopted,dog,senior,ps1120886,small,NaN,gray/blue/silver/salt & pepper,no shedding,NaN,NaN,NaN,NaT,900085001212610,NaN,400,NaN,NaN,2021-09-07,female,NaN,2010-06-02,"12 years, 7 months",NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,<p>abbey is a jack russell terrier and schnauz...,NaN,NaN,NaN,jade fucinaro- vetting coordinator,NaN,NaN,NaN,other,NaN,0,0
8,8,abby,schnauzer (miniature),,NaN,NaN,2018-10-02,24.0,2018-10-26,adopted,dog,senior,ps333886,small,NaN,gray,NaN,NaN,NaN,NaN,NaT,985112010405694,NaN,250,NaN,NaN,2021-09-07,female,NaN,2009-10-01,"13 years, 3 months",NaN,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,NaN,1.0,NaN,"<p>abby, the 9 year old miniature schnauzer is...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,other,NaN,0,0
10,10,abby,chihuahua,,NaN,NaN,2018-10-14,113.0,NaT,deceased,dog,adult,ps337595,small,NaN,"black, tan",NaN,NaN,NaN,NaN,2018-10-13,981020009722452,NaN,100,NaN,NaN,2021-09-07,female,NaN,2011-10-18,"11 years, 3 months",NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,1.0,NaN,1.0,1.0,<p>abby is a charming chihuahua who loves to s...,originally surrendered by homeless person. ad...,NaN,NaN,NaN,NaN,NaN,NaN,relinquished by owner,NaN,0,0
11,11,abby,labrador retriever,,NaN,NaN,2021-07-05,569.0,NaT,returned to owner,dog,senior,ps1149963,large,NaN,yellow,sheds a lot,NaN,NaN,NaN,NaT,NaN,NaN,265,NaN,NaN,2021-09-07,female,NaN,2008-09-05,"14 years, 4 months",NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,1.0,NaN,1.0,1.0,"<p>this senior sweetie is abby, a yellow lab b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,other,NaN,1,0


In [21]:
# Use this to view all rows in the series
pd.set_option('display.max_rows', None)
# Check what the adoption fee column looks like
muddy_p_df.adoption_fee

5                                                  600
6                                                  400
8                                                  250
10                                                 100
11                                                 265
12                                                 NaN
13                                                 200
15                                                 550
17                                                 465
18                                                 475
19                                                 400
20                                                 350
21                                                 400
22                                                 450
23                                                 400
24                                                 365
25                                                 400
26                                                 350
27        

In [22]:
# Turn off displaying max rows (this saves memory)
pd.reset_option('display.max_rows')

Several entries contain bonded pairs and show one adoption fee for two dogs. I need to remove those so I can later transform the adoption fee column to a float.

In [23]:
# Remove 'For the pair' and 'for both' and 'for ava and marley together' 'for bella and max together' 'for oddie and bogo together'
# 'for candy and charles together' 'for pair' 'for rosey and dude, together' '200-275' 'for both dogs together'
# 'for katy and hannah together' 'for jiggs and maggie together' 'for rusty and joey together' 'please contact paws for additonal information'
# 'for lucy and lacey together' 'for ruby and mambo together' 'for max and nike together' '375-400' 'for phineas and ferb together'
# 'for rosey and dude, together' 'for sage and hazel togethr'

In [24]:
# Add a new column for bonded pairs. This will allow me to keep potentially valuable information.
muddy_p_df.insert(24, 'bonded_pair', '')

In [25]:
# Check to make sure the column has been added
muddy_p_df.head(1)

,Unnamed: 0,pet_name,pet_breed,other_name,litter_name,parent_of_litter,date_pet_entered_your_care,length_of_stay_days,adoption_foster_date,current_status,pet_type,pet_age,petstablished_id,size,weight,color,shedding,coat_length,temperament,breed_type,date_placed_in_current_location,microchip_id,microchip_manufacturer,adoption_fee,bonded_pair,internal_id,animal_control_id,last_updated_at,gender,acquired_by,date_of_birth,age_in_years,declawed_status,coat_pattern,events_attendance,is_mix,shots_up_to_date,spayed_neutered,hypoallergenic,housebroken,special_need,gets_along_with_cats,gets_along_with_dogs,gets_along_with_kids,description,internal_notes,behavioral_tracking_notes,where_was_pet_originally_found,coordinators,additional_comments,owner_requested_euthanasia,additional_internal_comments,type_of_intake,siblings,courtesy_post,cross_post
5,5,aaliyah,australian shepherd,,NaN,NaN,2023-01-08,8.0,2023-01-16,adopted,dog,puppy,ps1615513,medium,NaN,"gray, black, white",sheds a lot,NaN,NaN,NaN,2023-01-17,900085001279038,NaN,600,,NaN,NaN,2023-01-17,female,NaN,2023-11-04,-283 days,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,<p>are you looking for an active pup to take w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relinquished by owner,NaN,0,0


In [26]:
# Show the current type of the adoption fee column
print('Old adoption fee type: ', muddy_p_df.adoption_fee.dtypes)
# Change the adoption fee column type to string (this will make it easier to work with later)
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.astype('string')
# Show the new type of the adoption fee column
print('New adoption fee type: ', muddy_p_df.adoption_fee.dtypes)

Old adoption fee type:  object
New adoption fee type:  string


In [27]:
# Show the current type of the bonded pair column
print('Old bonded pair type: ', muddy_p_df.bonded_pair.dtypes)
# Change the bonded pair column type to string
muddy_p_df.bonded_pair = muddy_p_df.bonded_pair.astype('string')
# Show the new type of the bonded pair column
print('New bonded pair type: ', muddy_p_df.bonded_pair.dtypes)

Old bonded pair type:  object
New bonded pair type:  string


In [28]:
# Fill the NaN values with an empty string - this will prevent errors with iterating through the rows
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.fillna('')

In [29]:
# If the adoption fee column contains an indication that an animal is part of a bonded pair, paste the value of adoption fee
# into the bonded pair column
muddy_p_df.bonded_pair = muddy_p_df.adoption_fee.apply(lambda row: row[0:row.find('for|both|and|together|togethr')] if ' ' in row else None)

In [30]:
# Check that the values of adoption fee have been transferred over to bonded pair
muddy_p_df.bonded_pair.iloc[60:70]

81               None
83               None
84               None
85               None
86               None
87    350 for the pai
88               None
90               None
91               None
93               None
Name: bonded_pair, dtype: object

I would like to change the bonded pair column to be strings with yes or no, which later could be changed to binary. 

In [31]:
# Replace NaN values in the bonded pair column with empty strings (prevents iteration errors later)
muddy_p_df.bonded_pair = muddy_p_df.bonded_pair.fillna('')

In [32]:
# Change the bonded_pair column rows to 'yes' if they contain strings that indicate bonded pair
muddy_p_df.loc[muddy_p_df.bonded_pair.str.contains('for|both|and|together|togethr'), 'bonded_pair'] = 'yes'

In [33]:
# Check to see if the strings have been changed
muddy_p_df.bonded_pair.iloc[60:70]

81       
83       
84       
85       
86       
87    yes
88       
90       
91       
93       
Name: bonded_pair, dtype: object

In [34]:
# Replace the empty strings with the word 'no' to indicate a dog is NOT part of a bonded pair
muddy_p_df.bonded_pair = muddy_p_df.bonded_pair.replace('', 'no')

In [35]:
# Check to see if the strings have been changed
muddy_p_df.bonded_pair.iloc[60:70]

81     no
83     no
84     no
85     no
86     no
87    yes
88     no
90     no
91     no
93     no
Name: bonded_pair, dtype: object

In [36]:
# Use .replace to change every no in the bonded pair column to 0 and every yes in the bonded pair column to 1
# no=0, yes=1
muddy_p_df.bonded_pair = muddy_p_df.bonded_pair.replace(to_replace = ['no', 'yes'], value = [0, 1])

In [37]:
# Check to see if the strings have been changed
muddy_p_df.bonded_pair.iloc[60:70]

81    0
83    0
84    0
85    0
86    0
87    1
88    0
90    0
91    0
93    0
Name: bonded_pair, dtype: int64

In [38]:
# Replace NaN values in the adoption fee column with empty strings (prevents iteration errors later)
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.fillna('')

In [39]:
# Split the row values into a list on spaces if there are spaces in the row (this captures any of the bonded pair indications)
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.apply(lambda row: row.split(' ') if ' ' in row else row)
# If the row type is a list (it had spaces and was changed above), keep only the first value in the list
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.apply(lambda row: row.pop(0) if type(row) == list else row)

In [40]:
# Check that the bonded pair strings have been removed
muddy_p_df.adoption_fee.iloc[60:70]

81    265
83    350
84    315
85    465
86    300
87    350
88    365
90    400
91    250
93    565
Name: adoption_fee, dtype: object

In [41]:
# There is at least one entry that contains something like '250-275'. I want to get rid of that and keep only one value
# Split the row values into a list on - if there is a - in the row
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.apply(lambda row: row.split('-') if '-' in row else row)
# If the row type is a list (it had - and was changed above), keep only the first value in the list
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.apply(lambda row: row.pop(0) if type(row) == list else row)

In [42]:
# There is an entry called 'please contact muddy paws...'. I want to get rid of that.
# Split the rows into a list on 'please' if they contain 'please'
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.apply(lambda row: row.split('please') if 'please' in row else row)
# If the row type is a list (it had 'please' and was changed avove), keep only the first value in the list (which should be blank)
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.apply(lambda row: row.pop(0) if type(row) == list else row)

In [43]:
# Check the adoption fee column again
muddy_p_df.adoption_fee.tail(10)

3543    300
3545    400
3547    350
3548    400
3549    350
3551    465
3553    150
3555    300
3556    400
3557    415
Name: adoption_fee, dtype: object

In [44]:
# Need to change the blank values to NaN in order to change column type to float
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.mask(muddy_p_df.adoption_fee == '')

In [45]:
# Show the current type of the adoption fee column
print('Old adoption fee type: ', muddy_p_df.adoption_fee.dtypes)
# Change the adoption fee column type to float
muddy_p_df.adoption_fee = muddy_p_df.adoption_fee.astype('float')
# Show the new type of the adoption fee column
print('New adoption fee type: ', muddy_p_df.adoption_fee.dtypes)

Old adoption fee type:  object
New adoption fee type:  float64


Age needs adjusted

In [46]:
# Check the age in years column
muddy_p_df.age_in_years

5                -283 days
6       12 years, 7 months
8       13 years, 3 months
10      11 years, 3 months
11      14 years, 4 months
               ...        
3551     1 years, 9 months
3553                   NaN
3555     9 years, 3 months
3556     3 years, 0 months
3557     1 years, 9 months
Name: age_in_years, Length: 2690, dtype: object

Notice that the age in years column contains strings, including years AND months on the column. This won't be easy to do math on, so I need to change that. My goal is to have two new columns: total age in years and total age in months.

In [47]:
# Add an column for the months values in the age in years column to be split into
muddy_p_df.insert(33, 'age_in_months', '')

In [48]:
# Show the current type of the age_in_years column
print('Old age in years type: ', muddy_p_df.age_in_years.dtypes)
# Change the type of the age_in_years column to string
muddy_p_df.age_in_years = muddy_p_df.age_in_years.astype('string')
# Show the new type of the age_in_years column
print('New age in years type: ', muddy_p_df.age_in_years.dtypes)

Old age in years type:  object
New age in years type:  string


In [49]:
# Fill the NaN values with blank strings (they can't be iterated through)
muddy_p_df.age_in_years = muddy_p_df.age_in_years.fillna('')

In [50]:
# If the word 'months' is in the age_in_years column, paste the whole value of age_in_years column to the new age_in_months column
muddy_p_df.age_in_months = muddy_p_df.age_in_years.apply(lambda row: row[0:row.find('months')] if 'months' in row else None)

In [51]:
# Check to make sure the values pasted over from the age_in_years column
muddy_p_df.age_in_months

5               None
6       12 years, 7 
8       13 years, 3 
10      11 years, 3 
11      14 years, 4 
            ...     
3551     1 years, 9 
3553            None
3555     9 years, 3 
3556     3 years, 0 
3557     1 years, 9 
Name: age_in_months, Length: 2690, dtype: object

In [52]:
# I want to remove the months portion from the age_in_years column
# Split age_in_years values to a list on the , if they have a ,
muddy_p_df.age_in_years = muddy_p_df.age_in_years.apply(lambda row: row.split(',') if ',' in row else row)
# Keep only the first value of the list if the row type is a list (changed above)
muddy_p_df.age_in_years = muddy_p_df.age_in_years.apply(lambda row: row.pop(0) if type(row) == list else row)

In [53]:
# Check the age_in_years column to ensure that months have been removed
muddy_p_df.age_in_years

5       -283 days
6        12 years
8        13 years
10       11 years
11       14 years
          ...    
3551      1 years
3553             
3555      9 years
3556      3 years
3557      1 years
Name: age_in_years, Length: 2690, dtype: object

In [54]:
# Check the age_in_months column - it shows that years have not yet been removed
muddy_p_df.age_in_months

5               None
6       12 years, 7 
8       13 years, 3 
10      11 years, 3 
11      14 years, 4 
            ...     
3551     1 years, 9 
3553            None
3555     9 years, 3 
3556     3 years, 0 
3557     1 years, 9 
Name: age_in_months, Length: 2690, dtype: object

In [55]:
# Fill the NaN values in age_in_months with an empty string (makes iterating possible)
muddy_p_df.age_in_months = muddy_p_df.age_in_months.fillna('')

In [56]:
# Split the age_in_months values into lists on the , if they contain ,
muddy_p_df.age_in_months = muddy_p_df.age_in_months.apply(lambda row: row.split(',') if ',' in row else row)
# Keep only the last value of the lists if the row type is a list (changed above)
muddy_p_df.age_in_months = muddy_p_df.age_in_months.apply(lambda row: row.pop(-1) if type(row) == list else row)

In [57]:
# Check age_in_months to be sure that the years values were removed
muddy_p_df.age_in_months

5          
6        7 
8        3 
10       3 
11       4 
       ... 
3551     9 
3553       
3555     3 
3556     0 
3557     9 
Name: age_in_months, Length: 2690, dtype: object

In [58]:
# Review the age_in_years column
muddy_p_df.age_in_years

5       -283 days
6        12 years
8        13 years
10       11 years
11       14 years
          ...    
3551      1 years
3553             
3555      9 years
3556      3 years
3557      1 years
Name: age_in_years, Length: 2690, dtype: object

In [59]:
# We can see above that there is a strange value (-283 days). I want to remove that value completely
# If there is a - in the row, split into a list on the -
muddy_p_df.age_in_years = muddy_p_df.age_in_years.apply(lambda row: row.split('-') if '-' in row else row)
# If the type of the row is a list (changed above) keep only the first value of the list (which in this case should be blank)
muddy_p_df.age_in_years = muddy_p_df.age_in_years.apply(lambda row: row.pop(0) if type(row) == list else row)

In [60]:
# Check to be sure the negative value was removed
muddy_p_df.age_in_years

5               
6       12 years
8       13 years
10      11 years
11      14 years
          ...   
3551     1 years
3553            
3555     9 years
3556     3 years
3557     1 years
Name: age_in_years, Length: 2690, dtype: object

In [61]:
# Remove the 'years' string from the age_in_years column
# Split into lists on the spaces if 'years' is in the row
muddy_p_df.age_in_years = muddy_p_df.age_in_years.apply(lambda row: row.split(' ') if 'years' in row else row)
# Keep only the first value of the list if the row type is a list (changed above)
muddy_p_df.age_in_years = muddy_p_df.age_in_years.apply(lambda row: row.pop(0) if type(row) == list else row)

In [62]:
# Check age_in_years to be sure 'years' was removed
muddy_p_df.age_in_years

5         
6       12
8       13
10      11
11      14
        ..
3551     1
3553      
3555     9
3556     3
3557     1
Name: age_in_years, Length: 2690, dtype: object

In [63]:
# Add a new column titled 'age_years_to_months'
# I will be using this column to calculate how many months the years in the age_in_years column are equal to
muddy_p_df.insert(34, 'age_years_to_months', '')

In [64]:
# Show the current type of the age_in_years column
print('Old age in years type: ', muddy_p_df.age_in_years.dtypes)
# Change the type of the age_in_years column to numerical values, coercing errors
muddy_p_df.age_in_years = pd.to_numeric(muddy_p_df.age_in_years, errors = 'coerce')
# Change the type of the age_in_years column to float values
muddy_p_df.age_in_years = muddy_p_df.age_in_years.astype('float')
# Sho the new type of the age_in_years column
print('New age in years type: ', muddy_p_df.age_in_years.dtypes)

Old age in years type:  object
New age in years type:  float64


In [65]:
# Check the age_in_years column
muddy_p_df.age_in_years

5        NaN
6       12.0
8       13.0
10      11.0
11      14.0
        ... 
3551     1.0
3553     NaN
3555     9.0
3556     3.0
3557     1.0
Name: age_in_years, Length: 2690, dtype: float64

In [66]:
# Set the age_years_to_months column to be equal to the age_in_years column * 12
muddy_p_df.age_years_to_months = muddy_p_df.age_in_years * 12

In [67]:
# Check the age_years_to_months column to verify that it has been changed
muddy_p_df.age_years_to_months

5         NaN
6       144.0
8       156.0
10      132.0
11      168.0
        ...  
3551     12.0
3553      NaN
3555    108.0
3556     36.0
3557     12.0
Name: age_years_to_months, Length: 2690, dtype: float64

In [68]:
# Show the current type of the age_in_months column
print('Old age in months type: ', muddy_p_df.age_in_months.dtypes)
# Change the type of the age_in_months column, coercing errors
muddy_p_df.age_in_months = pd.to_numeric(muddy_p_df.age_in_months, errors = 'coerce')
# Change the type of the age_in_months column to float
muddy_p_df.age_in_months = muddy_p_df.age_in_months.astype('float')
# Show the new type of the age_in_months column
print('New age in months type: ', muddy_p_df.age_in_months.dtypes)

Old age in months type:  object
New age in months type:  float64


In [69]:
# Add a new column called age_total_months to reflect the actual age of the dog in months
muddy_p_df.insert(34, 'age_total_months', '')

In [70]:
# Set the age_total_months column to be equal to age_years_to_months + age_in_months
muddy_p_df.age_total_months = muddy_p_df.age_years_to_months + muddy_p_df.age_in_months

In [71]:
# Check that the values in age_total_months make sense
muddy_p_df.age_total_months

5         NaN
6       151.0
8       159.0
10      135.0
11      172.0
        ...  
3551     21.0
3553      NaN
3555    111.0
3556     36.0
3557     21.0
Name: age_total_months, Length: 2690, dtype: float64

In [72]:
# Check the top of the dataframe to see the extra columns
muddy_p_df.head(2)

,Unnamed: 0,pet_name,pet_breed,other_name,litter_name,parent_of_litter,date_pet_entered_your_care,length_of_stay_days,adoption_foster_date,current_status,pet_type,pet_age,petstablished_id,size,weight,color,shedding,coat_length,temperament,breed_type,date_placed_in_current_location,microchip_id,microchip_manufacturer,adoption_fee,bonded_pair,internal_id,animal_control_id,last_updated_at,gender,acquired_by,date_of_birth,age_in_years,declawed_status,age_in_months,age_total_months,age_years_to_months,coat_pattern,events_attendance,is_mix,shots_up_to_date,spayed_neutered,hypoallergenic,housebroken,special_need,gets_along_with_cats,gets_along_with_dogs,gets_along_with_kids,description,internal_notes,behavioral_tracking_notes,where_was_pet_originally_found,coordinators,additional_comments,owner_requested_euthanasia,additional_internal_comments,type_of_intake,siblings,courtesy_post,cross_post
5,5,aaliyah,australian shepherd,,NaN,NaN,2023-01-08,8.0,2023-01-16,adopted,dog,puppy,ps1615513,medium,NaN,"gray, black, white",sheds a lot,NaN,NaN,NaN,2023-01-17,900085001279038,NaN,600.0,0,NaN,NaN,2023-01-17,female,NaN,2023-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,<p>are you looking for an active pup to take w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relinquished by owner,NaN,0,0
6,6,abbey,schnauzer (standard),,NaN,NaN,2021-06-02,43.0,2021-07-15,adopted,dog,senior,ps1120886,small,NaN,gray/blue/silver/salt & pepper,no shedding,NaN,NaN,NaN,NaT,900085001212610,NaN,400.0,0,NaN,NaN,2021-09-07,female,NaN,2010-06-02,12.0,NaN,7.0,151.0,144.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,<p>abbey is a jack russell terrier and schnauz...,NaN,NaN,NaN,jade fucinaro- vetting coordinator,NaN,NaN,NaN,other,NaN,0,0


In [73]:
# Create a new column (paralell to the age_total_months column) to contain the total dog age in years
muddy_p_df.insert(34, 'age_total_years', '')

In [74]:
# Create a age_months_to_years column (paralell to the age_years_to_months column) to calculate the months column to a years value
muddy_p_df.insert(34, 'age_months_to_years', '')

In [75]:
# Set age_months_to_years to match age_in_months divided by 12
muddy_p_df.age_months_to_years = muddy_p_df.age_in_months / 12

In [76]:
# Check that the values in age_months_to_years make sense
muddy_p_df.age_months_to_years

5            NaN
6       0.583333
8       0.250000
10      0.250000
11      0.333333
          ...   
3551    0.750000
3553         NaN
3555    0.250000
3556    0.000000
3557    0.750000
Name: age_months_to_years, Length: 2690, dtype: float64

In [77]:
# Set the age_total_years column to equal age_in_years + age_months_to_years
muddy_p_df.age_total_years = muddy_p_df.age_in_years + muddy_p_df.age_months_to_years

In [78]:
# Check that the values in age_total_years column make sense
muddy_p_df.age_total_years

5             NaN
6       12.583333
8       13.250000
10      11.250000
11      14.333333
          ...    
3551     1.750000
3553          NaN
3555     9.250000
3556     3.000000
3557     1.750000
Name: age_total_years, Length: 2690, dtype: float64

In [79]:
# Check the top of the dataframe to determine which columns need to be removed
muddy_p_df.head(2)

,Unnamed: 0,pet_name,pet_breed,other_name,litter_name,parent_of_litter,date_pet_entered_your_care,length_of_stay_days,adoption_foster_date,current_status,pet_type,pet_age,petstablished_id,size,weight,color,shedding,coat_length,temperament,breed_type,date_placed_in_current_location,microchip_id,microchip_manufacturer,adoption_fee,bonded_pair,internal_id,animal_control_id,last_updated_at,gender,acquired_by,date_of_birth,age_in_years,declawed_status,age_in_months,age_months_to_years,age_total_years,age_total_months,age_years_to_months,coat_pattern,events_attendance,is_mix,shots_up_to_date,spayed_neutered,hypoallergenic,housebroken,special_need,gets_along_with_cats,gets_along_with_dogs,gets_along_with_kids,description,internal_notes,behavioral_tracking_notes,where_was_pet_originally_found,coordinators,additional_comments,owner_requested_euthanasia,additional_internal_comments,type_of_intake,siblings,courtesy_post,cross_post
5,5,aaliyah,australian shepherd,,NaN,NaN,2023-01-08,8.0,2023-01-16,adopted,dog,puppy,ps1615513,medium,NaN,"gray, black, white",sheds a lot,NaN,NaN,NaN,2023-01-17,900085001279038,NaN,600.0,0,NaN,NaN,2023-01-17,female,NaN,2023-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,<p>are you looking for an active pup to take w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relinquished by owner,NaN,0,0
6,6,abbey,schnauzer (standard),,NaN,NaN,2021-06-02,43.0,2021-07-15,adopted,dog,senior,ps1120886,small,NaN,gray/blue/silver/salt & pepper,no shedding,NaN,NaN,NaN,NaT,900085001212610,NaN,400.0,0,NaN,NaN,2021-09-07,female,NaN,2010-06-02,12.0,NaN,7.0,0.583333,12.583333,151.0,144.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,<p>abbey is a jack russell terrier and schnauz...,NaN,NaN,NaN,jade fucinaro- vetting coordinator,NaN,NaN,NaN,other,NaN,0,0


In [80]:
# Drop the columns that are no longer needed
muddy_p_df = muddy_p_df.drop(['age_in_years', 'age_in_months', 'age_months_to_years', 'age_years_to_months'], axis = 1)

In [81]:
# Check that the columns have been dropped
muddy_p_df.head(2)

,Unnamed: 0,pet_name,pet_breed,other_name,litter_name,parent_of_litter,date_pet_entered_your_care,length_of_stay_days,adoption_foster_date,current_status,pet_type,pet_age,petstablished_id,size,weight,color,shedding,coat_length,temperament,breed_type,date_placed_in_current_location,microchip_id,microchip_manufacturer,adoption_fee,bonded_pair,internal_id,animal_control_id,last_updated_at,gender,acquired_by,date_of_birth,declawed_status,age_total_years,age_total_months,coat_pattern,events_attendance,is_mix,shots_up_to_date,spayed_neutered,hypoallergenic,housebroken,special_need,gets_along_with_cats,gets_along_with_dogs,gets_along_with_kids,description,internal_notes,behavioral_tracking_notes,where_was_pet_originally_found,coordinators,additional_comments,owner_requested_euthanasia,additional_internal_comments,type_of_intake,siblings,courtesy_post,cross_post
5,5,aaliyah,australian shepherd,,NaN,NaN,2023-01-08,8.0,2023-01-16,adopted,dog,puppy,ps1615513,medium,NaN,"gray, black, white",sheds a lot,NaN,NaN,NaN,2023-01-17,900085001279038,NaN,600.0,0,NaN,NaN,2023-01-17,female,NaN,2023-11-04,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,<p>are you looking for an active pup to take w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relinquished by owner,NaN,0,0
6,6,abbey,schnauzer (standard),,NaN,NaN,2021-06-02,43.0,2021-07-15,adopted,dog,senior,ps1120886,small,NaN,gray/blue/silver/salt & pepper,no shedding,NaN,NaN,NaN,NaT,900085001212610,NaN,400.0,0,NaN,NaN,2021-09-07,female,NaN,2010-06-02,NaN,12.583333,151.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,<p>abbey is a jack russell terrier and schnauz...,NaN,NaN,NaN,jade fucinaro- vetting coordinator,NaN,NaN,NaN,other,NaN,0,0


In [82]:
# Reorder the columns in a sequence that makes sense to me
# All the binary variables are together
# The most important information is first
# The text information is last
muddy_p_df = muddy_p_df.reindex(['pet_name', 'pet_breed', 'other_name', 'litter_name', 'length_of_stay_days', 'current_status', 
                                 'pet_age', 'age_total_years', 'age_total_months', 'adoption_fee', 'size', 'weight', 'gender', 
                                 'siblings', 'type_of_intake', 'acquired_by', 'where_was_pet_originally_found', 'coordinators', 
                                 'pet_type', 'parent_of_litter', 'color', 'shedding', 'coat_length', 'coat_pattern', 
                                 'events_attendance', 'temperament', 'breed_type', 'courtesy_post', 'cross_post', 'bonded_pair', 
                                 'declawed_status', 'is_mix', 'shots_up_to_date', 'spayed_neutered', 'hypoallergenic', 
                                 'housebroken', 'special_need', 'gets_along_with_cats', 'gets_along_with_dogs', 
                                 'gets_along_with_kids', 'owner_requested_euthanasia', 'date_pet_entered_your_care', 
                                 'adoption_foster_date', 'date_placed_in_current_location', 'last_updated_at', 'date_of_birth', 
                                 'petstablished_id', 'microchip_id', 'microchip_manufacturer', 'internal_id', 
                                 'animal_control_id',  'description', 'internal_notes', 'behavioral_tracking_notes', 
                                 'additional_comments',  'additional_internal_comments'], axis = 1)

In [83]:
# Check that the columns have been changed
muddy_p_df.head(2)

,pet_name,pet_breed,other_name,litter_name,length_of_stay_days,current_status,pet_age,age_total_years,age_total_months,adoption_fee,size,weight,gender,siblings,type_of_intake,acquired_by,where_was_pet_originally_found,coordinators,pet_type,parent_of_litter,color,shedding,coat_length,coat_pattern,events_attendance,temperament,breed_type,courtesy_post,cross_post,bonded_pair,declawed_status,is_mix,shots_up_to_date,spayed_neutered,hypoallergenic,housebroken,special_need,gets_along_with_cats,gets_along_with_dogs,gets_along_with_kids,owner_requested_euthanasia,date_pet_entered_your_care,adoption_foster_date,date_placed_in_current_location,last_updated_at,date_of_birth,petstablished_id,microchip_id,microchip_manufacturer,internal_id,animal_control_id,description,internal_notes,behavioral_tracking_notes,additional_comments,additional_internal_comments
5,aaliyah,australian shepherd,,NaN,8.0,adopted,puppy,NaN,NaN,600.0,medium,NaN,female,NaN,relinquished by owner,NaN,NaN,NaN,dog,NaN,"gray, black, white",sheds a lot,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,NaN,2023-01-08,2023-01-16,2023-01-17,2023-01-17,2023-11-04,ps1615513,900085001279038,NaN,NaN,NaN,<p>are you looking for an active pup to take w...,NaN,NaN,NaN,NaN
6,abbey,schnauzer (standard),,NaN,43.0,adopted,senior,12.583333,151.0,400.0,small,NaN,female,NaN,other,NaN,NaN,jade fucinaro- vetting coordinator,dog,NaN,gray/blue/silver/salt & pepper,no shedding,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,NaN,2021-06-02,2021-07-15,NaT,2021-09-07,2010-06-02,ps1120886,900085001212610,NaN,NaN,NaN,<p>abbey is a jack russell terrier and schnauz...,NaN,NaN,NaN,NaN


In [84]:
# Get an overview of the types in the dataframe
muddy_p_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2690 entries, 5 to 3557
Data columns (total 56 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   pet_name                         2688 non-null   object        
 1   pet_breed                        2690 non-null   object        
 2   other_name                       2690 non-null   object        
 3   litter_name                      616 non-null    object        
 4   length_of_stay_days              2484 non-null   float64       
 5   current_status                   2690 non-null   object        
 6   pet_age                          2601 non-null   object        
 7   age_total_years                  2328 non-null   float64       
 8   age_total_months                 2328 non-null   float64       
 9   adoption_fee                     2547 non-null   float64       
 10  size                             2590 non-null   object     

In [85]:
# Remove columns that contain all null values
muddy_p_df = muddy_p_df.drop(['owner_requested_euthanasia', 'animal_control_id', 'additional_internal_comments', 'declawed_status'], axis = 1)

In [86]:
muddy_p_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2690 entries, 5 to 3557
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   pet_name                         2688 non-null   object        
 1   pet_breed                        2690 non-null   object        
 2   other_name                       2690 non-null   object        
 3   litter_name                      616 non-null    object        
 4   length_of_stay_days              2484 non-null   float64       
 5   current_status                   2690 non-null   object        
 6   pet_age                          2601 non-null   object        
 7   age_total_years                  2328 non-null   float64       
 8   age_total_months                 2328 non-null   float64       
 9   adoption_fee                     2547 non-null   float64       
 10  size                             2590 non-null   object     

In [87]:
muddy_p_df.to_excel('part_2_mp_cleaning.xlsx')

Notes:

- The size column is having issues, it is returning "'numpy.int32' object has no attribute"
    - Need to fix this somehow, but that will come later
- The types may need to be changed for the purposes of analysis, but for now they will be left as is